In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

In [2]:
xy_df = pd.read_csv('./data/iris.csv', delimiter=',')
x_data = xy_df.iloc[:,0:-1].to_numpy()
x_data.shape

(150, 4)

In [3]:
 pd.get_dummies(xy_df.iloc[:,-1]).values.argmax(1).astype(np.float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
      dtype=float32)

In [4]:
# Creating Network

class Model(torch.nn.Module):
    
    def __init__(self):
        """
        Initialize the network with the layers
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(4, 10)
        self.l2 = torch.nn.Linear(10, 8)
        self.l3 = torch.nn.Linear(8, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        """
        Defining the forward pass of the data based on the layers created above
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

In [5]:
# Creating the Dataset and Dataloader

class MyDataset(Dataset):
    
    def __init__(self):
        xy_df = pd.read_csv('./data/iris.csv', delimiter=',')
        self.len = xy_df.shape[0]
        self.x_data = torch.from_numpy(xy_df.iloc[:,0:-1].to_numpy(dtype=np.float32))
        self.y_data = torch.from_numpy((pd.get_dummies(xy_df.iloc[:,-1])).values.argmax(1).astype(np.float32))
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len       

In [6]:
model = Model()
my_dataset = MyDataset()

In [7]:
train_loader = DataLoader(dataset = my_dataset, batch_size=16, shuffle=True, num_workers=0)

In [8]:
criterion = torch.nn.BCELoss(reduction='mean')
optimus = torch.optim.SGD(model.parameters(), lr = 0.05)

In [9]:
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        # getting the inputs and labels
        inputs, labels = data
#         print(f'labels: {labels.shape}')
        # Predictions from the model
        y_pred = model(inputs)
#         print(labels.shape)
        
        # Get the loss
        loss = criterion(y_pred, labels)
        print(f'Epoch: {epoch}, Loss: {loss.item()}')
        
        # optimus roll out
        optimus.zero_grad()
        loss.backward()
        optimus.step()
        

C:\Users\abhis\Anaconda3\envs\fastai\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0, Loss: 0.5405045747756958
Epoch: 0, Loss: 0.4360932409763336
Epoch: 0, Loss: 0.5628055930137634
Epoch: 0, Loss: 0.41424405574798584
Epoch: 0, Loss: 0.3848228454589844
Epoch: 0, Loss: 0.2688494324684143
Epoch: 0, Loss: 0.4241332709789276
Epoch: 0, Loss: 0.5201851725578308
Epoch: 0, Loss: 0.3002222776412964
Epoch: 0, Loss: 0.3375948369503021
Epoch: 1, Loss: 0.07833589613437653
Epoch: 1, Loss: 0.29924270510673523
Epoch: 1, Loss: 0.5644407272338867
Epoch: 1, Loss: 0.2867211103439331
Epoch: 1, Loss: 0.059754449874162674
Epoch: 1, Loss: 0.48803412914276123
Epoch: 1, Loss: 0.02433396875858307
Epoch: 1, Loss: 0.07684843987226486
Epoch: 1, Loss: 0.6535183191299438
Epoch: 1, Loss: 0.23174096643924713


C:\Users\abhis\Anaconda3\envs\fastai\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([6, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
